In [1]:
try:
    import src.IAFautoclass.IAFautoclass_GUI as IAFautoclass_GUI

    # Create GUI object
    gui = IAFautoclass_GUI.IAFautoclass_GUI()
    gui.display_gui()
except FileNotFoundError:
    print(".env file with variables missing.")



Image(value=b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x01,\x00\x00\x00\x80\x08\x06\x00\x00\x00\xb1\xd4TT\x…

Label(value='*** Welcome to IAF automatic classification! ***')

Text(value='default', description='Project name:', description_tooltip='Enter a distinct project name for the …

Dropdown(description='Databases:', description_tooltip='These are the databases of choice', options=('', 'Admi…

Dropdown(description='Tables:', description_tooltip='These are the database tables of choice', disabled=True, …

...

Label(value='Class summary: N/A')

Button(button_style='success', description='Continue', disabled=True, icon='check', style=ButtonStyle(), toolt…

Box(children=(Dropdown(description='Reduction:', description_tooltip='Pick by what method the number of featur…

Button(button_style='success', description='Start', disabled=True, icon='check', style=ButtonStyle(), tooltip=…

Output(layout=Layout(border='2px solid grey'))